# MBAM model reduction for the MWC model of BK
Model taken from Aldrich 2002 (Equation 1) <br>
Initial parameters taken from Miranda 2013 PNAS, Suppl fig 5 <br>
Algorithm as in Transtrum 2015 (Equation 49-51) - except for signs

The code below iteratively finds limits (boundaries) and stiff nonlinear combinations of sloppy parameters for the MWC model of BK, while preserving the model output function $P_{open}(V, [Ca] \ | \ \theta)$.

The iterative model reduction proceeds as follows:
+ $\theta = \{L_0, z_L, J_0, z_J, K_d, C, D, E \}$
    * $z_L -> 0$
    
    
+ $\phi^1 = \{L_0, J_0, z_J, K_d, C, D, E \}$
    * $L_0^{1/4} \rightarrow 0, \quad C \rightarrow \infty, \quad D \rightarrow \infty, \quad E \rightarrow 0 $
    

+ $\phi^2 = \{J_0, z_J, K_d, L_0^{1/4}C, L_0^{1/4}D, L_0^{-1/4}E \}$ 
    * $L_0^{1/4}C \rightarrow 0, \quad L_0^{-1/4}E \rightarrow \infty $
    
    
+ $\phi^3 = \{J_0, z_J, K_d, CE, L_0^{1/4}D \}$ 
    * $J_0 \rightarrow 0, \quad L_0^{1/4}D \rightarrow \infty $
 
 
+ $\phi^4 = \{J_0L_0^{1/4}D, z_J, K_d, CE  \}$ 
    * $K_d \rightarrow \infty, \quad CE \rightarrow \infty $
 
 
+ $\phi^5 = \{J_0L_0^{1/4}D, z_J, CEK_d^{-1}  \}$ 

The $P_{open}(V, [Ca] \ | \ \phi^5)$ function is an excellent approximation of the original (way below experimental error), despite having only 3 parameters. Each of the parameters are meaningful, the first one describing the slope of the voltage sensitivity, the second the middle point of the voltage sensitivity and the third the calcium sensitivity.
Note that the 4 parameter-model ($\phi^4$) describes both the voltage and the calcium sensitivity curves as two-parameter sigmoids. The allosteric MWC model introduces nonlinear interaction between these two sensitivity curves.

In [2]:
using NBInclude #Includes for ipynb-s
#using JLD # Julia data I/O
nbinclude("MBAM.ipynb")
nbinclude("BK_functions/BK_functions.ipynb")


BK_cost (generic function with 1 method)

In [3]:
# Create grid to evaluate over
using Iterators
Ca=([0.0, 0.7, 4.0, 12.0, 22.0, 55.0, 70.0, 95.0]*1e-6)
V=collect(-100:25:200)*1e-3
Ca = float(Ca)
V = float(V)

x_grid = Array(Any, (length(Ca)*length(V),))
iterind = 0;
for j1 in product(Ca, V)
    iterind += 1;
    x_grid[iterind] = collect(j1)
end
size(x_grid[1])

(2,)

In [ ]:
# Single MBAM call that runs through all iterations of BK model reduction with NON-AUTOMATED model reduction
phi0 = [2.2e-6, 0.42, 0.1026, 0.58, 39*1e-6, 6.16,30.4,2.0];
outp = MBAM(BK_simulator, phi0, x_grid, model_iters=[10, 11, 12, 13, 14], 
move_dir=[-1, -1, 1, 1, -1], boundary_time=10, verbose=1, reduce_func=BK_reduce);



  0.716084 seconds (665.10 k allocations: 31.533 MB, 0.78% gc time)
Model iteration 10, Single g_cost call takes: 
  0.060158 seconds (76.52 k allocations: 3.465 MB)
Model iteration 10, initial values:
      D = [  2.45e-05,   2.83e-04,   1.32e-03,   4.91e-02,   8.41e-02,   3.34e-01,   5.48e+00,   2.81e+01]